In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3

from keras.applications.xception import preprocess_input as xception_preprocessor
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor

from sklearn.metrics import log_loss, accuracy_score
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import regularizers

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

/Users/junyan/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load training data

In [98]:
LABELS = "/Users/junyan/Downloads/Homework/DeepLearning/Project/data/labels.csv"

train_df = pd.read_csv(LABELS)

top_breeds = sorted(list(train_df['breed'].value_counts().head(120).index))
train_df = train_df[train_df['breed'].isin(top_breeds)]


In [4]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split

SEED = 1234

TRAIN_FOLDER = "/Users/junyan/Downloads/Homework/DeepLearning/Project/data/train/"
TEST_FOLDER = "/Users/junyan/Downloads/Homework/DeepLearning/Project/data/test/"

DIM = 299

train_df['image_path'] = train_df.apply( lambda x: ( TRAIN_FOLDER + x["id"] + ".jpg" ), axis=1 )

train_data = np.array([ img_to_array(load_img(img, target_size=(DIM, DIM))) for img in train_df['image_path'].values.tolist()]).astype('float32')
train_labels = train_df['breed']


x_train, x_validation, y_train, y_validation = train_test_split(train_data, train_labels, test_size=0.2, stratify=np.array(train_labels), random_state=SEED)


In [5]:
x_train.shape

(8177, 299, 299, 3)

One hot encoded

In [6]:
y_train = pd.get_dummies(y_train.reset_index(drop=True), columns=top_breeds).as_matrix()
y_validation = pd.get_dummies(y_validation.reset_index(drop=True), columns=top_breeds).as_matrix()

print(y_train[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0]


Load pretrained model's weight

In [8]:
from os import makedirs
from os.path import expanduser, exists, join

!ls /Users/junyan/Downloads/Homework/DeepLearning/Project/data/pretrained-model/

cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)
    
!cp /Users/junyan/Downloads/Homework/DeepLearning/Project/data/pretrained-model/*notop* ~/.keras/models/
!cp /Users/junyan/Downloads/Homework/DeepLearning/Project/data/pretrained-model/imagenet_class_index.json ~/.keras/models/
!cp /Users/junyan/Downloads/Homework/DeepLearning/Project/data/pretrained-model/resnet50* ~/.keras/models/

Kuszma.JPG.zip
imagenet_class_index.json
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5.zip
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5.zip
inception_v3_weights_tf_dim_ordering_tf_kernels.h5.zip
inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5.zip
resnet50_weights_tf_dim_ordering_tf_kernels.h5.zip
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5.zip
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5.zip
xception_weights_tf_dim_ordering_tf_kernels.h5.zip
xception_weights_tf_dim_ordering_tf_kernels_notop.h5.zip


# Build models and train

In [106]:
models = {
    #"InceptionV3": {
    #    "model": InceptionV3,
    #    "preprocessor": inception_v3_preprocessor,
    #    "input_shape": (299,299,3),
    #    "seed": 1234,
    #    "pooling": "avg"
    #},
    "Xception": {
        "model": Xception,
        "preprocessor": xception_preprocessor,
        "input_shape": (299,299,3),
        "seed": 5512,
        "pooling": "avg"
    }
}

for model_name, model in models.items():
    print("Predicting : {}".format(model_name))
    filename = model_name + '_features.npy'
    validfilename = model_name + '_validfeatures.npy'
    if exists(filename):
        features = np.load(filename)
        validation_features = np.load(validfilename)
    else:
        #image propocessing:
        train_datagen = ImageDataGenerator(
                zoom_range = 0.3,
                width_shift_range=0.1,
                height_shift_range=0.1)
        validation_datagen = ImageDataGenerator()
        

        # Use pretrained model to generate Bottleneck Features:
        Xception_model = Xception(weights='imagenet', include_top=False, input_shape = (299,299,3), pooling="avg")
        
        train_datagen.preprocessing_function = xception_preprocessor
        train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=batch_size, seed = 5512)
        features = Xception_model.predict_generator(train_generator,verbose=1)
        
        validation_datagen.preprocessing_function = xception_preprocessor
        validation_generator = validation_datagen.flow(x_validation, y_validation, shuffle=False, batch_size=batch_size, seed = 5512)
        validation_features = Xception_model.predict_generator(validation_generator,verbose=1)
        
        np.save(filename, features)
        np.save(validfilename, validation_features)
        
      
    
    # Bulid top layers:
    inputs = Input(shape=(2048,))
    #x = Dense(256, activation='relu')(inputs)
    #x = BatchNormalization()(x)
    #x = Dropout(0.5)(x)
    predictions = Dense(120, activation='softmax')(inputs)
    
    model_top = Model(inputs = inputs, outputs = predictions)
    model_top.summary()
    
    model_top.compile(optimizer = Adam(0.0001), 
                      loss = categorical_crossentropy, 
                      metrics = ['accuracy'])
    
    # Train the top model:
    model_top.fit(features, y_train,
                batch_size=256, epochs=150, verbose=1, 
                validation_data=(validation_features, y_validation))
    


Predicting : Xception
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 2048)              0         
_________________________________________________________________
dense_56 (Dense)             (None, 120)               245880    
Total params: 245,880
Trainable params: 245,880
Non-trainable params: 0
_________________________________________________________________
Train on 8177 samples, validate on 2045 samples
Epoch 1/150
8177/8177 [==============================] - 2s 223us/step - loss: 4.6404 - acc: 0.0525 - val_loss: 4.4067 - val_acc: 0.1746
Epoch 2/150
8177/8177 [==============================] - 0s 40us/step - loss: 4.2643 - acc: 0.2923 - val_loss: 4.0198 - val_acc: 0.5081
Epoch 3/150
8177/8177 [==============================] - 0s 41us/step - loss: 3.9053 - acc: 0.5626 - val_loss: 3.6473 - val_acc: 0.7183
Epoch 4/150
8177/8177 [==============================

Epoch 56/150
8177/8177 [==============================] - 0s 40us/step - loss: 0.5140 - acc: 0.8892 - val_loss: 0.3594 - val_acc: 0.9042
Epoch 57/150
8177/8177 [==============================] - 0s 39us/step - loss: 0.5099 - acc: 0.8896 - val_loss: 0.3570 - val_acc: 0.9046
Epoch 58/150
8177/8177 [==============================] - 0s 40us/step - loss: 0.5059 - acc: 0.8905 - val_loss: 0.3548 - val_acc: 0.9046
Epoch 59/150
8177/8177 [==============================] - 0s 39us/step - loss: 0.5019 - acc: 0.8915 - val_loss: 0.3528 - val_acc: 0.9042
Epoch 60/150
8177/8177 [==============================] - 0s 41us/step - loss: 0.4981 - acc: 0.8923 - val_loss: 0.3504 - val_acc: 0.9051
Epoch 61/150
8177/8177 [==============================] - 0s 38us/step - loss: 0.4945 - acc: 0.8924 - val_loss: 0.3485 - val_acc: 0.9051
Epoch 62/150
8177/8177 [==============================] - 0s 40us/step - loss: 0.4908 - acc: 0.8932 - val_loss: 0.3467 - val_acc: 0.9032
Epoch 63/150
8177/8177 [=================

8177/8177 [==============================] - 0s 38us/step - loss: 0.3715 - acc: 0.9221 - val_loss: 0.3041 - val_acc: 0.9037
Epoch 115/150
8177/8177 [==============================] - 0s 38us/step - loss: 0.3699 - acc: 0.9217 - val_loss: 0.3042 - val_acc: 0.9037
Epoch 116/150
8177/8177 [==============================] - 0s 38us/step - loss: 0.3684 - acc: 0.9221 - val_loss: 0.3037 - val_acc: 0.9042
Epoch 117/150
8177/8177 [==============================] - 0s 39us/step - loss: 0.3669 - acc: 0.9227 - val_loss: 0.3036 - val_acc: 0.9037
Epoch 118/150
8177/8177 [==============================] - 0s 40us/step - loss: 0.3653 - acc: 0.9232 - val_loss: 0.3032 - val_acc: 0.9046
Epoch 119/150
8177/8177 [==============================] - 0s 39us/step - loss: 0.3639 - acc: 0.9226 - val_loss: 0.3029 - val_acc: 0.9042
Epoch 120/150
8177/8177 [==============================] - 0s 38us/step - loss: 0.3624 - acc: 0.9241 - val_loss: 0.3030 - val_acc: 0.9027
Epoch 121/150
8177/8177 [=======================

# Load testing data and predict

In [16]:
#loading test images
from tqdm import tqdm
import cv2
import os
from keras.preprocessing import image

TEST_FOLDER = '/Users/junyan/Downloads/Homework/DeepLearning/Project/data/sample_submission.csv'
test_data = pd.read_csv(TEST_FOLDER)

x_test = []
for i in tqdm(test_data['id'].values):
    img = cv2.imread('/Users/junyan/Downloads/Homework/DeepLearning/Project/data/test/{}.jpg'.format(i))
    x_test.append(cv2.resize(img, (299, 299)))
    
x_test = np.array(x_test, np.float32)
print x_test.shape

100%|██████████| 10357/10357 [00:40<00:00, 256.08it/s]


(10357, 299, 299, 3)


In [107]:
testfilename = 'Xception_testfeatures.npy'
if exists(testfilename):
        test_features = np.load(testfilename)
else:
    test_datagen = ImageDataGenerator()
    test_datagen.preprocessing_function = xception_preprocessor
    test_generator = test_datagen.flow(x_test, shuffle=False, batch_size=batch_size, seed = 5512)
    test_features = Xception_model.predict_generator(test_generator, verbose=1)
    np.save('Xception_testfeatures.npy', test_features)

In [108]:
predictions = model_top.predict(test_features)
print predictions.shape

(10357, 120)


In [109]:
#generating Kaggle upload file
df_train = pd.read_csv('/Users/junyan/Downloads/Homework/DeepLearning/Project/data/labels.csv')
targets_series = df_train['breed']
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [110]:
#generating Kaggle upload file
test_DF = pd.read_csv('/Users/junyan/Downloads/Homework/DeepLearning/Project/data/sample_submission.csv')
sub = pd.DataFrame(predictions)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', test_DF['id'])
sub.head(5)
print(sub.shape)
sub.to_csv('Xception_sub2.csv', sep=',',index=False)

(10357, 121)
